In [83]:
# Libraries import
import torch
import torch.nn as nn
import torch.nn.functional as func
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence

import random

import pandas as pd
import numpy as np

import os
import sys

import math

import time

# GPU
if torch.cuda.is_available():
    print("GPU is running.")
    dev = "cuda"
else:
    print("CPU is running.")
    dev = "cpu"

device = torch.device(dev)


GPU is running.


In [84]:
# Dataset 
dataset_all_rows_total = pd.read_csv('data/data_main.tsv',sep='\t')

count = 0
tfs = []

tfs_id = []

strength_sequence = []

full_sequence = []

dataset_all_rows = dataset_all_rows_total.sample(frac=0.02, random_state= 42)

# seprating original sequence, motifs, positions

for index, sequence in dataset_all_rows.iterrows():

    motif_id = []


    for col_index in range(1, len(sequence) -3): 
        
        if pd.notna(sequence[col_index]): 
        
            motif_id.append(col_index)  

    tfs_id.append(motif_id)

    sequence = np.array(sequence.dropna().values)

    full_sequence.append(sequence[0])

    count+=1

    motif_id = []
    
    tfs.append(sequence[1:-3])

    strength_sequence.append(sequence[-2])


/tmp/ipykernel_938478/2045651676.py:2: DtypeWarning: Columns (13,63,75,93,94,98) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_all_rows_total = pd.read_csv('data/data_main.tsv',sep='\t')


In [85]:

# Separating the motifs and position
all_motifs = []
all_positions = []

for seq_index, motifs in enumerate(tfs):

    motifs_sequence = []
    position_motifs = []

    for each_item in motifs:

        each_motif = each_item.split(':')[0]

        each_motif_position = int(each_item.split(':')[1])

        motifs_sequence.append(each_motif)

        position_motifs.append(each_motif_position)            

    all_motifs.append(motifs_sequence)

    all_positions.append(position_motifs)


# Arranging motifs in order

sorted_motifs_list = []

sorted_positions = []
full_sequence_all = []

sorted_class_id = []

for index in range(len(all_motifs)):

    temp_zip = list(zip(all_motifs[index], all_positions[index], tfs_id[index]))

    if not temp_zip:  # Skip empty cases
        continue

    sorted_zip = sorted(temp_zip, key=lambda x: x[1])

    sorted_motifs, sorted_position, sorted_id = zip(*sorted_zip)

    sorted_motifs_list.append(sorted_motifs)
    sorted_positions.append(sorted_position)

    sorted_class_id.append(sorted_id)

    full_sequence_all.append(full_sequence[index])


# Filtering empty tokens if present
strength_sequence_filtered = []


def removeEmptyMotifs(motifs_list):

    index = 0


    filtered_motifs = []

    for item in motifs_list:

        if len(item) > 0:

            filtered_motifs.append(item)

            strength_sequence_filtered.append(strength_sequence[index])

        index+=1

    return filtered_motifs


def removeEmpty(motifs_list):

    filtered_motifs = []

    for item in motifs_list:


        if len(item) > 0:

            filtered_motifs.append(item)

    return filtered_motifs

filtered_tokens = removeEmptyMotifs(sorted_motifs_list)

position_sequences = removeEmpty(sorted_positions)

strength_sequence = strength_sequence_filtered 

In [86]:
# Encoding sequences using one-hot encoding
encoding = {
    'A': [0, 0],
    'C': [0, 1],
    'G': [1, 0],
    'T': [1, 1]
}

def one_hot_encoding(motifs_sequences):

    all_motifs_encoded = []

    for seq in motifs_sequences:

        encoded = []

        for each_motif in seq:

            encoded_seq = []

            for base in each_motif:

                if base in encoding:

                    encoded_seq.extend(encoding[base])

            encoded.append(''.join(map(str, encoded_seq)))

        all_motifs_encoded.append(encoded)

    return all_motifs_encoded


encoded_motifs_list = one_hot_encoding(filtered_tokens)


In [87]:
# Generating embedding of the encoded information

encoded_motifs_list_all = []

for seq in encoded_motifs_list:

    motifs_list = []

    for motif in seq:


        
        motif_tensor = torch.tensor([int(bit) for bit in motif], dtype=torch.float32)


        input_dim = len(motif)

        linear = nn.Linear(input_dim, 50)


        motif_tensor = motif_tensor.view(1, -1)

        output = linear(motif_tensor)

        output = output.reshape(-1,)

        motifs_list.append(output.detach())

    encoded_motifs_list_all.append(motifs_list)




In [88]:

# Filtering sequence = []

def filterSequenceByPosition(encoded_motifs_list, all_positions, original_motif_sequences):
    sorted_vector_representations = []
    sorted_positions = []
    sorted_original_words = []
    sorted_class_ids = []

    for index in range(len(encoded_motifs_list)):

        temp_zip = list(zip(encoded_motifs_list[index],all_positions[index], original_motif_sequences[index]))

        sorted_zip = sorted(temp_zip,key=lambda x: x[1])

        sorted_vec,sorted_position,sorted_word = zip(*sorted_zip)

        sorted_vector_representations.append(sorted_vec)
        sorted_positions.append(sorted_position)
        sorted_original_words.append(sorted_word)
        # sorted_class_ids.append(class_ids)

    return sorted_vector_representations,sorted_positions,sorted_original_words

sorted_motif_encoding_representations , sorted_motif_positions, sorted_original_motifs = filterSequenceByPosition(encoded_motifs_list_all, position_sequences, filtered_tokens)


In [ ]:
# Making training and testing seqeunces

all_indices = [val for val in range(len(sorted_motif_encoding_representations))]
train_indices = random.sample(all_indices, int(0.8 * len(all_indices)))
test_indices = list(set(all_indices).difference(set(train_indices)))

training_vec_sequences = [
    torch.stack([sub_seq.to(dtype=torch.float32, device=device) for sub_seq in sorted_motif_encoding_representations[index]])
    for index in train_indices
]

training_sequences_positions = [torch.from_numpy(np.array(sorted_motif_positions[index])).to(dtype=torch.long, device=device) for index in train_indices]
training_motifs_sequences = [sorted_original_motifs[index] for index in train_indices]
training_sequence_strength= [torch.from_numpy(np.array(strength_sequence[index])).to(dtype=torch.float32, device=device) for index in train_indices]

testing_vec_sequences = [
    torch.stack([sub_seq.to(dtype=torch.float32, device=device) for sub_seq in sorted_motif_encoding_representations[index]])
    for index in test_indices
]

testing_sequences_positions = [torch.from_numpy(np.array(sorted_motif_positions[index])).to(dtype=torch.long, device=device) for index in test_indices]
testing_motifs_sequences = [sorted_original_motifs[index] for index in test_indices]
testing_sequence_strength = [torch.from_numpy(np.array(strength_sequence[index])).to(dtype=torch.float32, device=device) for index in test_indices]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
# Preparing the data batches

# Get data batches
def getBatches(vec_data, original_data, positions, outputs,batch_size = 1):
    counter = 0

    vec_data_batches = []
    original_data_batches = []
    position_data_batches = []
    output_data_batches = []

    while counter < len(vec_data):
        batched_vec_data = vec_data[counter: counter+batch_size][0]
        batched_original_data = original_data[counter:counter+batch_size][0]
        batched_position_data = positions[counter:counter+batch_size][0]
        batched_output_data = outputs[counter: counter+batch_size][0]

        counter = counter + batch_size

        vec_data_batches.append(batched_vec_data)
        original_data_batches.append(batched_original_data)
        position_data_batches.append(batched_position_data)
        output_data_batches.append(batched_output_data)

    return (vec_data_batches, original_data_batches, position_data_batches, output_data_batches)

# Training batches

seq_training_batches, training_motifs_batches, trainining_motifs_positions_batches, training_strength_batches = getBatches(training_vec_sequences, training_motifs_sequences, training_sequences_positions, training_sequence_strength)

# Testing batches

seq_testing_batches, testing_motifs_batches, testing_motifs_positions_batches, testing_strength_batches = getBatches(testing_vec_sequences, testing_motifs_sequences, testing_sequences_positions, testing_sequence_strength)



In [ ]:
# Positional embedding

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000, negative_dist = 250, positive_dist = 50):
        super().__init__()

        self.dropout = nn.Dropout(p=dropout)

        self.negative_dist = negative_dist
        self.d_model = d_model

        self.seq_length = negative_dist + positive_dist

        position = torch.arange(0,self.seq_length,device = device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)).to(device=device)

        pe = torch.zeros(self.seq_length, 1, d_model).to(device=device)
        pe[:, 0, 0::2] = torch.sin(position * div_term).to(device=device)
        pe[:, 0, 1::2] = torch.cos(position * div_term).to(device=device)

        self.register_buffer('pe', pe)

    def forward(self, x, positions):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        batch_size, seq_len , embedding_dim = x.shape

        x = x.transpose(1,0)

        indices = positions.reshape(-1) + self.negative_dist

        x = x + self.pe[indices]

        x = self.dropout(x)

        x.transpose(1,0)

        return x


In [33]:
class GeneFormer(nn.Module):
    def __init__(self, input_dim = 50, model_dim = 512, n_output_heads = 1, nhead = 4, enc_layers_count = 3, motif_count = 1):
        super().__init__()

        self.input_dim = input_dim
        self.model_dim = model_dim

        self.motif_count = motif_count

        self.n_output_heads = n_output_heads

        self.nhead = nhead
        self.enc_layers_count = enc_layers_count

        # Linear Layers for the input.

        self.input_embedding_1 = torch.nn.Linear(self.input_dim, int((self.model_dim)/2))
        self.input_embedding_2 = torch.nn.Linear(int((self.model_dim)/2), self.model_dim)

        self.positional_embedding = PositionalEncoding(d_model=self.model_dim)

        # Transformer model definition.
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=self.model_dim, nhead=self.nhead)
        self.encoders = torch.nn.TransformerEncoder(self.encoder_layer , num_layers = self.enc_layers_count)

        # Final output layer for the model.

        self.genetics_classifier_1 = torch.nn.Linear(self.model_dim, int((self.model_dim /2)))

        self.genetics_classifier_2 = torch.nn.Linear(int((self.model_dim /2)), self.n_output_heads)

        self.activation_relu = torch.nn.ReLU()

        # Dropout Functions
        self.dropout_5 = torch.nn.Dropout(p = 0.05)
        self.dropout_10 = torch.nn.Dropout(p = 0.10)



    def forward(self,inputs, positions):

        batch_size, seq_length, features = inputs.shape

        embedded_input = self.input_embedding_1(inputs)

        embedded_input = self.dropout_5(embedded_input)

        embedded_input = self.input_embedding_2(embedded_input)

        embedded_input = self.dropout_5(embedded_input)

        embedded_input = embedded_input.reshape(batch_size,seq_length,self.model_dim)

        positional_embedded_input = self.positional_embedding(embedded_input, positions)

        encoded_x = self.encoders(positional_embedded_input)

        encoded_x = torch.mean(encoded_x, dim = 1)

        encoded_x = torch.mean(encoded_x, dim = 0)

        output_x = self.genetics_classifier_1(encoded_x)

        output_x = self.activation_relu(output_x)

        output_x = self.genetics_classifier_2(output_x)

        return output_x.reshape(-1,)



Epoch Number: 1 Loss : 68424.30831447463
Epoch Number: 2 Loss : 68171.03461568187
Epoch Number: 3 Loss : 68059.3068385293
Epoch Number: 4 Loss : 67992.92851136725
Time Elapsed: 68.97268748283386
Training Epoch Losses: [68424.30831447463, 68171.03461568187, 68059.3068385293, 67992.92851136725]


In [ ]:
# Model configuration

gene_model = GeneFormer(input_dim = 50, model_dim = 512, n_output_heads= 1, nhead= 4 , enc_layers_count= 4, motif_count = 1)

gene_model = gene_model.to(device=device)

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(gene_model.parameters(), lr= 5e-5)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3 ,gamma = 0.6, last_epoch= -1, verbose=False)



In [ ]:
# Training loop

def TrainModel(seq_representation_inputs, motifs_inputs, position_inputs, strength_output, epoch_number):

    total_loss = 0
    total_batches = 0
    batch_size = 1

    for seq_index , batch_input in enumerate(seq_representation_inputs):

        optimizer.zero_grad()

        total_batches +=1

        position_input = position_inputs[seq_index].reshape(1,-1,1).to(device)
        actual_output = strength_output[seq_index].reshape(-1).to(device)
        tokens, features = batch_input.shape
        batch_input = batch_input.reshape(batch_size,tokens,features)

        output = gene_model(batch_input, position_input)
        loss = criterion(output,actual_output).to(device)
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()
    print('Epoch Number: {} Loss : {}'.format(epoch_number, total_loss / (total_batches)))
    return total_loss / (total_batches)

# Training parameters

epoch_number = 1
epoch_end_count = 0
train_epoch_avg_losses_sp = []

start_time = time.time()

gene_model.train(True)



while epoch_number <= 4:

    temp_holder = list(zip(seq_training_batches, training_motifs_batches, trainining_motifs_positions_batches, training_strength_batches))
    random.shuffle(temp_holder)
    seq_training_batches, training_motifs_batches, trainining_motifs_positions_batches, training_strength_batches = zip(*temp_holder)

    epoch_loss_sp, weight_change_classfier, weight_chage_encoder = TrainModel(seq_training_batches, training_motifs_batches, trainining_motifs_positions_batches, training_strength_batches, epoch_number)

    epoch_number += 1
    if epoch_number % 2 == 0:

        checkpoint = {
        'epoch': epoch_number,  
        'model_state_dict': gene_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }
        torch.save(checkpoint, '/local/data/sdahal_p/genome-test/model_checkpoint_terminal.pth')

    train_epoch_avg_losses_sp.append(epoch_loss_sp)

    scheduler.step()

end_time = time.time()

print("Time Elapsed:", end_time - start_time)
print("Training Epoch Losses:", train_epoch_avg_losses_sp)


In [30]:
# Testing starts

gene_model.eval()

def TestGeneModel(seq_representation_inputs, motifs_inputs, position_inputs, strength_output):

    actual_strength_values = []

    predicted_strength_values = []

    total_batches = 0

    with torch.no_grad():

        for input_index , batch_input in enumerate(seq_representation_inputs):

            total_batches +=1

            batch_size = 1

            position_input = position_inputs[input_index].reshape(1,-1,1)

            actual_output = strength_output[input_index].reshape(-1)

            tokens, features = batch_input.shape

            batch_input = batch_input.reshape(batch_size,tokens,features)

            output = gene_model(batch_input, position_input).to(device)

            actual_strength_values.append(actual_output)

            predicted_strength_values.append(output)

    return actual_strength_values, predicted_strength_values


actual_values, predicted_values = TestGeneModel(seq_testing_batches, testing_motifs_batches, testing_motifs_positions_batches, testing_strength_batches)


In [31]:
# Saving test results in csv file

import csv

print((actual_values[1]))
print(predicted_values[1])


import torch
actual_values = torch.tensor(actual_values, device='cuda:1')
predicted_values = torch.tensor(predicted_values, device='cuda:1')


actual_values = actual_values.cpu().numpy() 
predicted_values = predicted_values.cpu().numpy()

data = {
    "Actual Values": actual_values,
    "Predicted Values": predicted_values
}
df = pd.DataFrame(data)

file_name = "/local/data/sdahal_p/genome-test/result/strength_values.csv"

df.to_csv(file_name, index=False)


tensor([239.], device='cuda:0')
tensor([1.0020], device='cuda:0')


In [19]:
# loading for testing

checkpoint = torch.load('model_checkpoint.pth')
gene_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
